In [2]:
#import dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time
import lxml
import pymongo

In [3]:
#set up executable path and create splinter browser to load pages
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [4]:
#visit nasa's mars news site
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(1)

In [5]:
#scrape latest article title and description
html = browser.html
soup = BeautifulSoup(html,'html.parser')

article_list = soup.find("ul",class_="item_list")
news_title = article_list.find("div",class_="content_title").text
news_p = article_list.find("div",class_="article_teaser_body").text
print(news_title)
print(news_p)

NASA's Mars Rover Drivers Need Your Help
Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.


In [6]:
#visit jpl's mars images site and navigate to featured image
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
time.sleep(1)
browser.find_by_css('div.carousel_items a.fancybox').click()
time.sleep(1)

In [7]:
#scrape featured image url
html = browser.html
soup = BeautifulSoup(html,'html.parser')
featured_image = soup.find('img',class_='fancybox-image')
featured_image_url = "https://www.jpl.nasa.gov" + str(featured_image).split(" ")[2].split('"')[1]
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17470_ip.jpg'

In [17]:
#scrape weather from tweet of mars weather twitter account
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(1)
mars_weather = browser.find_by_css('div.js-tweet-text-container p.tweet-text').text
mars_weather

ElementDoesNotExist: no elements could be found with css "div.js-tweet-text-container p.tweet-text"

In [ ]:
#scrape table from mars facts
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html,'lxml')
table = soup.find_all('table')[0]

mars_facts_df_list = pd.read_html(str(table))

for dictionary in mars_facts_df_list:
    dictionary["description"] = dictionary.pop(0)
    dictionary["value"] = dictionary.pop(1)
mars_facts_dict_list = mars_facts_df_list[0].to_json(orient='records')

In [10]:
url = "https://space-facts.com/mars/"
mars_facts_df = pd.read_html(url)[0]
mars_facts_df = mars_facts_df.rename(index=str, columns={0: "Description", 1: "Value"})
mars_facts_html = mars_facts_df.to_html(index='False',index_names='False')
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [18]:
mars = {
    "news_title": news_title,
    "news_p": news_p,
    "featured_image_url": featured_image_url,
    "mars_weather": mars_weather,
    "mars_facts": mars_facts_html,
    "mars_hemispheres" :mars_hemi_html
}

NameError: name 'mars_weather' is not defined

In [12]:
#insert scraped info into mars_collection 
#now app.py only has to update values of collection

In [13]:
client = pymongo.MongoClient()
db = client.mars_db
collection = db.mars_collection

In [21]:
#clear out contents of db before inserting
db.collection.delete_many({})

In [22]:
db.collection.insert_one(mars_data)

NameError: name 'mars_data' is not defined

In [20]:
print(list(db.collection.find()))

[]
